In [1]:
!pip install -q -U datasets transformers rouge_score nltk

  Preparing metadata (setup.py) ... done


In [2]:
!pip install -q opendatasets

In [3]:
import opendatasets as od

In [4]:
od.download("https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: afreenhossain
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset


100%|██████████| 62.5M/62.5M [00:00<00:00, 1.28GB/s]

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("/content/resume-dataset/Resume/Resume.csv")

In [7]:
df.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [8]:
from transformers import AutoTokenizer

In [11]:
model_ckpt = "google-bert/bert-base-uncased"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [14]:
tokenizer("Learning to fine tune BERT for resume role classification!")

{'input_ids': [101, 4083, 2000, 2986, 8694, 14324, 2005, 13746, 2535, 5579, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.decode([101, 4083, 2000, 2986, 8694, 14324, 2005, 13746, 2535, 5579, 999, 102])

'[CLS] learning to fine tune bert for resume role classification! [SEP]'

In [17]:
df.drop(columns=["ID", "Resume_html"], inplace=True)

In [18]:
df.head(3)

,Resume_str,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR


### prepare data for training, testing and dataloader

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
train, test = train_test_split(df, test_size=0.3, random_state=42, stratify=df["Category"])

In [23]:
train.shape, test.shape

((1738, 2), (746, 2))

In [25]:
test, validation = train_test_split(test, random_state=42, stratify=test["Category"])

In [26]:
test.shape, validation.shape

((559, 2), (187, 2))

In [27]:
from datasets import DatasetDict, Dataset

In [29]:
dataset = DatasetDict({
    "train": Dataset.from_pandas(train, preserve_index=False),
    "test": Dataset.from_pandas(test, preserve_index=False),
    "validation": Dataset.from_pandas(validation, preserve_index=False)
})

In [30]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Resume_str', 'Category'],
        num_rows: 1738
    })
    test: Dataset({
        features: ['Resume_str', 'Category'],
        num_rows: 559
    })
    validation: Dataset({
        features: ['Resume_str', 'Category'],
        num_rows: 187
    })
})

### tokenizing batches

In [34]:
dataset["train"]["Resume_str"][0]

"         SALES       Summary     Focused and dedicated insurance professional motivated to provide superior customer service and drive sales through cold calling, referrals and strategic outside marketing activities.         Highlights          Self-motivated  Driven  Clear and articulate phone manner  Cheerful and energetic  Dependable        Property and casualty products  Exceptional interpersonal skills  Excellent communication skills  Strong sales-closer  Cold calling            Accomplishments     Consistently qualify for sales bonuses by exceeding sales targets by items as well as monetary goals.         Experience      Sales   02/2015   to   12/2015     Company Name   City  ,   State      Exceeded sales goals to 24,000 in one month's time.  Generated new leads weekly.  Contacted new and existing customers to discuss how their needs could be met with specific products and services.  Responded to all customer inquiries in a timely manner.  Retained  policyholders during annual r

In [35]:
def tokenize_data(batch):
  return tokenizer(batch["Resume_str"], padding=True, truncation=True)

In [38]:
tokenize_data(dataset["train"][0])

{'input_ids': [101, 4341, 12654, 4208, 1998, 4056, 5427, 2658, 12774, 2000, 3073, 6020, 8013, 2326, 1998, 3298, 4341, 2083, 3147, 4214, 1010, 6523, 7941, 2015, 1998, 6143, 2648, 5821, 3450, 1012, 11637, 2969, 1011, 12774, 5533, 3154, 1998, 2396, 24153, 3042, 5450, 18350, 1998, 18114, 12530, 3085, 3200, 1998, 19844, 3688, 11813, 6970, 28823, 4813, 6581, 4807, 4813, 2844, 4341, 1011, 3553, 3147, 4214, 17571, 10862, 7515, 2005, 4341, 29563, 2011, 17003, 4341, 7889, 2011, 5167, 2004, 2092, 2004, 12194, 3289, 1012, 3325, 4341, 6185, 1013, 2325, 2000, 2260, 1013, 2325, 2194, 2171, 2103, 1010, 2110, 14872, 4341, 3289, 2000, 2484, 1010, 2199, 1999, 2028, 3204, 1005, 1055, 2051, 1012, 7013, 2047, 5260, 4882, 1012, 11925, 2047, 1998, 4493, 6304, 2000, 6848, 2129, 2037, 3791, 2071, 2022, 2777, 2007, 3563, 3688, 1998, 2578, 1012, 5838, 2000, 2035, 8013, 27050, 1999, 1037, 23259, 5450, 1012, 6025, 3343, 17794, 2076, 3296, 14524, 2558, 1012, 10263, 17464, 7846, 2000, 4553, 2055, 2037, 3361, 3791, 19

In [39]:
dataset_encoded = dataset.map(tokenize_data, batch_size=None, batched=True)

Map:   0%|          | 0/1738 [00:00<?, ? examples/s]

Map:   0%|          | 0/559 [00:00<?, ? examples/s]

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

In [40]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['Resume_str', 'Category', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1738
    })
    test: Dataset({
        features: ['Resume_str', 'Category', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 559
    })
    validation: Dataset({
        features: ['Resume_str', 'Category', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 187
    })
})

In [70]:
len(df["Category"].unique())

24

In [61]:
print(dataset_encoded["train"][0].keys())

dict_keys(['Resume_str', 'Category', 'input_ids', 'token_type_ids', 'attention_mask'])


In [69]:
len(set(dataset_encoded["train"]["Category"]))

24

In [71]:
set(dataset_encoded["train"]["Category"])

{'ACCOUNTANT',
 'ADVOCATE',
 'AGRICULTURE',
 'APPAREL',
 'ARTS',
 'AUTOMOBILE',
 'AVIATION',
 'BANKING',
 'BPO',
 'BUSINESS-DEVELOPMENT',
 'CHEF',
 'CONSTRUCTION',
 'CONSULTANT',
 'DESIGNER',
 'DIGITAL-MEDIA',
 'ENGINEERING',
 'FINANCE',
 'FITNESS',
 'HEALTHCARE',
 'HR',
 'INFORMATION-TECHNOLOGY',
 'PUBLIC-RELATIONS',
 'SALES',
 'TEACHER'}

In [72]:
categories = list(set(dataset_encoded["train"]["Category"]))

In [77]:
label2id = {val: key for key, val in enumerate(categories)}

In [78]:
label2id

{'SALES': 0,
 'AVIATION': 1,
 'FINANCE': 2,
 'DESIGNER': 3,
 'ACCOUNTANT': 4,
 'ENGINEERING': 5,
 'ARTS': 6,
 'INFORMATION-TECHNOLOGY': 7,
 'BPO': 8,
 'BUSINESS-DEVELOPMENT': 9,
 'APPAREL': 10,
 'BANKING': 11,
 'AUTOMOBILE': 12,
 'CHEF': 13,
 'TEACHER': 14,
 'HR': 15,
 'CONSTRUCTION': 16,
 'ADVOCATE': 17,
 'HEALTHCARE': 18,
 'PUBLIC-RELATIONS': 19,
 'FITNESS': 20,
 'DIGITAL-MEDIA': 21,
 'AGRICULTURE': 22,
 'CONSULTANT': 23}

In [79]:
id2label = {key: val for key, val in enumerate(categories)}

In [80]:
id2label

{0: 'SALES',
 1: 'AVIATION',
 2: 'FINANCE',
 3: 'DESIGNER',
 4: 'ACCOUNTANT',
 5: 'ENGINEERING',
 6: 'ARTS',
 7: 'INFORMATION-TECHNOLOGY',
 8: 'BPO',
 9: 'BUSINESS-DEVELOPMENT',
 10: 'APPAREL',
 11: 'BANKING',
 12: 'AUTOMOBILE',
 13: 'CHEF',
 14: 'TEACHER',
 15: 'HR',
 16: 'CONSTRUCTION',
 17: 'ADVOCATE',
 18: 'HEALTHCARE',
 19: 'PUBLIC-RELATIONS',
 20: 'FITNESS',
 21: 'DIGITAL-MEDIA',
 22: 'AGRICULTURE',
 23: 'CONSULTANT'}

### BERT model

In [81]:
from transformers import AutoModel

In [82]:
model = AutoModel.from_pretrained(model_ckpt)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [83]:
model.config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.55.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [84]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [85]:
from transformers import AutoModelForSequenceClassification

In [86]:
num_labels = len(label2id)

In [87]:
num_labels

24

In [88]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [103]:
from transformers import AutoConfig

In [104]:
config = AutoConfig.from_pretrained(
    "google-bert/bert-base-uncased",
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label
)

In [105]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "SALES",
    "1": "AVIATION",
    "2": "FINANCE",
    "3": "DESIGNER",
    "4": "ACCOUNTANT",
    "5": "ENGINEERING",
    "6": "ARTS",
    "7": "INFORMATION-TECHNOLOGY",
    "8": "BPO",
    "9": "BUSINESS-DEVELOPMENT",
    "10": "APPAREL",
    "11": "BANKING",
    "12": "AUTOMOBILE",
    "13": "CHEF",
    "14": "TEACHER",
    "15": "HR",
    "16": "CONSTRUCTION",
    "17": "ADVOCATE",
    "18": "HEALTHCARE",
    "19": "PUBLIC-RELATIONS",
    "20": "FITNESS",
    "21": "DIGITAL-MEDIA",
    "22": "AGRICULTURE",
    "23": "CONSULTANT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "ACCOUNTANT": 4,
    "ADVOCATE": 17,
    "AGRICULTURE": 22,
    "APPAREL": 10,
    "ARTS": 6,
    "AUTOMOBILE": 1

In [106]:
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased",
    config=config
).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [107]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [108]:
model.classifier

Linear(in_features=768, out_features=24, bias=True)

### training

In [112]:
from transformers import TrainingArguments, Trainer

In [113]:
batch_size = 64

In [120]:
def encode_labels(batch):
  batch["labels"] = label2id[batch["Category"]]
  return batch

In [121]:
dataset_encoded = dataset_encoded.map(encode_labels)

Map:   0%|          | 0/1738 [00:00<?, ? examples/s]

Map:   0%|          | 0/559 [00:00<?, ? examples/s]

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

In [123]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [124]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    processing_class=tokenizer
)

In [125]:
trainer.train()

Step,Training Loss
500,1.448700


TrainOutput(global_step=654, training_loss=1.2254537529901628, metrics={'train_runtime': 553.3674, 'train_samples_per_second': 9.422, 'train_steps_per_second': 1.182, 'total_flos': 1372132025155584.0, 'train_loss': 1.2254537529901628, 'epoch': 3.0})

In [164]:
predictions = trainer.predict(dataset_encoded["test"])

In [165]:
predictions

PredictionOutput(predictions=array([[-1.6942273e-01, -1.8540943e-01,  5.7213831e+00, ...,
        -1.0411237e+00, -1.3400713e-01,  5.7590695e-04],
       [ 6.2613748e-02, -7.1825886e-01, -8.1756836e-01, ...,
        -3.4825811e-01, -1.3648050e-01, -9.2252684e-01],
       [-5.3641610e-02, -6.8814701e-01, -6.0401451e-02, ...,
        -6.5290749e-01, -1.7173652e-01,  2.4238102e-01],
       ...,
       [ 3.2532749e-01,  1.7341919e+00, -3.4575969e-01, ...,
         3.3663461e-01,  6.2592924e-01, -4.6068110e-02],
       [-6.0579550e-01, -1.5454505e-01,  1.2007648e+00, ...,
        -1.0762627e+00, -2.7515268e-01, -2.9988989e-02],
       [-8.6706334e-01,  8.7905221e-04,  4.5063093e-01, ...,
        -8.4202880e-01, -6.3104713e-01, -8.6496525e-02]], dtype=float32), label_ids=array([ 2, 14,  8, 17, 23, 18, 19,  7, 16, 17,  0, 14,  9, 11, 15, 20, 12,
       10,  9, 15, 12, 23, 10, 21,  9, 14, 23,  9, 23,  1, 22, 11,  6,  0,
       18, 16, 15,  3,  2, 13, 19, 20, 13,  3, 14, 14,  8, 19,  3,  0, 16,

In [166]:
predictions.metrics

{'test_loss': 0.7822864651679993,
 'test_runtime': 16.886,
 'test_samples_per_second': 33.104,
 'test_steps_per_second': 4.145}

In [133]:
predictions[0]

array([[-1.6942273e-01, -1.8540943e-01,  5.7213831e+00, ...,
        -1.0411237e+00, -1.3400713e-01,  5.7590695e-04],
       [ 6.2613748e-02, -7.1825886e-01, -8.1756836e-01, ...,
        -3.4825811e-01, -1.3648050e-01, -9.2252684e-01],
       [-5.3641610e-02, -6.8814701e-01, -6.0401451e-02, ...,
        -6.5290749e-01, -1.7173652e-01,  2.4238102e-01],
       ...,
       [ 3.2532749e-01,  1.7341919e+00, -3.4575969e-01, ...,
         3.3663461e-01,  6.2592924e-01, -4.6068110e-02],
       [-6.0579550e-01, -1.5454505e-01,  1.2007648e+00, ...,
        -1.0762627e+00, -2.7515268e-01, -2.9988989e-02],
       [-8.6706334e-01,  8.7905221e-04,  4.5063093e-01, ...,
        -8.4202880e-01, -6.3104713e-01, -8.6496525e-02]], dtype=float32)

In [134]:
predictions[0][0]

array([-1.6942273e-01, -1.8540943e-01,  5.7213831e+00, -5.9493971e-01,
        7.5849134e-01, -1.9165581e+00, -1.1706722e+00, -7.5697809e-01,
        4.5165997e-02,  4.4918546e-01, -1.1591309e+00,  1.2891805e+00,
        1.5376332e-01, -2.2571602e-01, -7.9027867e-01, -1.4905586e+00,
       -2.4276523e-01, -4.4367377e-02,  2.6958755e-01, -7.1395379e-01,
       -5.0599623e-01, -1.0411237e+00, -1.3400713e-01,  5.7590695e-04],
      dtype=float32)

In [135]:
len(predictions[0][0])

24

In [136]:
import numpy as np

In [138]:
np.argmax(predictions[0][0], axis=-1)

np.int64(2)

In [143]:
predictions.label_ids

array([ 2, 14,  8, 17, 23, 18, 19,  7, 16, 17,  0, 14,  9, 11, 15, 20, 12,
       10,  9, 15, 12, 23, 10, 21,  9, 14, 23,  9, 23,  1, 22, 11,  6,  0,
       18, 16, 15,  3,  2, 13, 19, 20, 13,  3, 14, 14,  8, 19,  3,  0, 16,
        4, 22, 12,  4, 14, 18, 17, 20, 10,  7,  2, 14,  4, 20, 16,  2, 19,
       21, 20, 10,  3, 11,  0, 14,  2, 13, 18, 19,  2, 17,  9, 21,  6, 10,
        6,  4,  2,  9, 22, 11,  4, 23, 19,  6, 10,  0,  4, 15, 19,  5, 18,
        1,  7,  5,  2,  5, 17, 11,  4,  5, 18, 13,  1, 14,  2, 11, 22,  1,
       19,  1, 22, 10,  0, 11,  6, 11,  4, 23,  8, 23, 11, 17, 16, 17, 18,
       18, 21, 18,  7, 20, 21, 17, 12,  3, 14,  6, 23, 15, 13, 15,  9, 21,
       20, 18, 19, 23, 16, 18,  9, 21,  3,  4, 11, 23, 17, 14,  3, 21,  7,
       12, 23,  9,  2,  3, 23,  5,  9,  9,  2,  1, 14, 13,  5, 18,  3, 11,
        1,  9,  3,  2, 10, 16, 16, 23, 19,  2,  9, 13, 15,  0,  7,  9, 23,
       13, 11,  0, 17, 11,  7,  2,  2, 21,  9,  3,  1,  0,  3, 21, 13, 16,
        2, 14, 16,  7, 13

In [144]:
len(predictions.predictions), len(predictions.label_ids)

(559, 559)

In [145]:
from sklearn.metrics import f1_score, accuracy_score

In [151]:
def compute_metrics(preds):
  labels = preds.label_ids
  predictions = preds.predictions.argmax(-1)
  f1 = f1_score(labels, predictions, average="weighted")
  accuracy = accuracy_score(labels, predictions)

  return {
      "f1_score": f1,
      "accuracy": accuracy
  }

In [152]:
print(compute_metrics(predictions))

{'f1_score': 0.800260970511122, 'accuracy': 0.8121645796064401}


In [155]:
train_preds = trainer.predict(dataset_encoded["train"])

In [156]:
print(compute_metrics(train_preds))

{'f1_score': 0.9041214714886608, 'accuracy': 0.9131185270425777}


In [157]:
val_preds = trainer.predict(dataset_encoded["validation"])

print(compute_metrics(val_preds))

{'f1_score': 0.8973262466706284, 'accuracy': 0.9090909090909091}


### predictions on custom data

In [167]:
preds = predictions.predictions.argmax(-1)

In [168]:
preds

array([ 2, 14, 18, 17, 10, 18, 19,  7, 16, 17,  0, 14,  9, 11, 15, 20,  6,
       21,  9, 15,  7, 23, 15,  1,  9, 14, 23,  9, 23,  1, 22, 11,  6,  0,
       18, 16, 15,  3,  2, 13, 11, 20, 13,  3, 14, 14,  9, 19,  3,  0, 16,
        4, 22,  1,  4, 14, 17, 17, 20,  3,  7,  2, 14,  4, 20, 16,  2, 19,
        7, 19, 10,  3,  0,  0, 14,  2, 13, 18, 19,  2, 11,  9, 21,  6, 10,
        6,  4,  2,  9, 22, 11,  4, 23, 20,  6, 21,  0,  4, 15, 15,  5, 19,
        1,  7,  5,  2,  5, 17, 23,  4,  5, 20,  9, 10, 14,  2, 11, 11,  1,
       19,  1, 11,  3,  0, 11,  1, 23,  4, 23, 11, 23, 11, 17, 16, 17, 18,
       18, 21, 18,  7, 20, 21, 17, 10,  3, 14,  6, 23, 15, 13, 15,  9, 21,
       20, 18, 21, 21, 16, 18,  9, 21,  3,  4, 11, 23, 17, 14,  3, 21,  7,
       15, 23,  9,  2,  3, 23,  5,  9,  9,  2, 10, 14, 13,  5, 18,  3, 11,
        1,  9,  3,  2, 10, 16, 16, 23, 19,  2,  9, 13, 15,  0,  7,  9, 23,
       13, 11,  0, 17, 11,  7,  2,  2,  6,  9,  3,  1,  0,  3, 21, 13,  3,
        2, 14, 16,  7, 13

In [170]:
id2label[preds[0]]

'FINANCE'

In [171]:
text = """
Resume

Name: Priya Sharma

Work Experience:

HR Manager | ABC Tech Solutions | Jan 2021 – Present
- Managed recruitment process across departments, reducing time-to-hire by 20%.
- Implemented employee wellness and engagement programs, increasing employee satisfaction scores by 15%.
- Conducted training workshops for managers on leadership and conflict resolution.

HR Executive | XYZ Pvt Ltd | Jun 2018 – Dec 2020
- Handled onboarding process for over 150 new employees annually.
- Assisted in payroll processing and benefits administration.
- Coordinated exit interviews and provided reports to improve retention strategies.

------------------------------------------------------------

Education:
MBA in Human Resources Management – Christ University, Bangalore (2018)
BBA – HR Specialization – Delhi University (2016)

------------------------------------------------------------

Skills:
- Recruitment & Talent Acquisition
- HR Policies & Compliance
- Employee Engagement
- Payroll & Benefits Management
- Training & Development
- HRMS & ATS Tools

------------------------------------------------------------

Certifications:
- SHRM-CP (Society for Human Resource Management – Certified Professional)
- Certified Talent Acquisition Specialist (CTAS)

------------------------------------------------------------

Languages:
- English (Fluent)
- Hindi (Native)
"""


In [172]:
input_encoded = tokenizer(text, return_tensors="pt").to(device)

In [174]:
outputs = model(**input_encoded)

In [175]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.3888, -0.1175, -0.7351, -1.0652, -0.1916, -0.6767,  0.4576, -0.5430,
          0.2282,  0.0874,  0.1038, -0.1385,  0.1492, -0.1374, -0.3253,  5.3484,
         -0.5960,  0.9969, -1.2780,  1.2064, -0.1991,  0.7048, -0.4765,  0.0917]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [176]:
logits = outputs.logits

In [177]:
torch.argmax(logits, dim=-1).item()

15

In [178]:
id2label[torch.argmax(logits, dim=-1).item()]

'HR'

In [188]:
def get_prediction(text):
  with torch.no_grad():
      input_encoded = tokenizer(text, return_tensors="pt").to(device)
      outputs = model(**input_encoded)
      logits = outputs.logits

  return id2label[torch.argmax(logits, dim=-1).item()]

In [191]:
banking = """
         MANAGEMENT CONSULTANT         Skills        Microsoft Office Suite
   Factset
   Ipreo              Experience     07/2013   to   12/2014     Management Consultant    Company Name   －   City  ,   State      Utilized management abilities and knowledge of coding to oversee a team of 10 software engineers tasked
   with designing a back office database for a multi-billion dollar client
   Managed cross-functional teams in a high demand environment to complete a global strategy project under
   strenuous deadlines
   Drafted proposals for client engagements highlighting the expectations of service, engagement pricing and
   associated costs
   Evaluated CCAR vendor models and developed proprietary probability of default models for a multi-billion
   dollar mortgage portfolio
   Created valuation models designed to value both public and private equity for tax reporting purposes.         11/2010   to   02/2011     Investment Banking Analyst    Company Name   －   City  ,   State      Assisted lead associate and vice president in facilitating merger and acquisition transactions in the life
    science industry by reviewing and analyzing pro forma financial statements and updating models
    accordingly

"""

get_prediction(sales)

'BANKING'

In [192]:
agri = """

**Resume**

**Name:** Harish Kumar
**Email:** harish.kumar@example.com

**Professional Summary:**
Agriculture specialist with 12+ years of experience in sustainable farming practices, crop management, and agri-tech solutions.

**Work Experience:**
- Agricultural Consultant at AgroTech (2016–Present)
- Farm Manager at GreenFields (2010–2016)

**Education:** B.Sc in Agriculture

**Skills:** Crop Management, Irrigation Systems, Sustainable Farming, Agri-Tech Tools
"""

get_prediction(agri)

'AGRICULTURE'

In [193]:
fitness = """

**Resume**

**Name:** Varun Joshi
**Email:** varun.joshi@example.com

**Professional Summary:**
Certified fitness trainer with 5+ years of experience in personal training and nutrition coaching. Helped 200+ clients achieve health goals.

**Work Experience:**
- Fitness Trainer at Cult.Fit (2019–Present)
- Personal Trainer at Gold’s Gym (2015–2019)

**Education:** Diploma in Fitness and Nutrition

**Skills:** Personal Training, Nutrition Plans, Strength Training, Yoga

"""

get_prediction(fitness)

'FITNESS'

In [194]:
aviation = """
**Resume**

**Name:** Arjun Mehta
**Email:** arjun.mehta@example.com

**Professional Summary:**
Certified commercial pilot with 7+ years of aviation experience. Skilled in aircraft operations, flight planning, and ensuring compliance with aviation safety standards.

**Work Experience:**
- Pilot at Indigo Airlines (2018–Present)
- Co-Pilot at Air India (2015–2018)

**Education:** B.Sc in Aviation

**Skills:** Aircraft Operations, Flight Planning, Aviation Safety, Navigation Systems
"""

get_prediction(aviation)

'AVIATION'

In [195]:
sales = """
**Resume**

**Name:** Riya Kapoor
**Email:** riya.kapoor@example.com

**Professional Summary:**
Sales executive with 5+ years of experience in B2B and retail sales. Skilled in client acquisition, negotiation, and closing high-value deals. Consistently exceeded quarterly sales targets by 20%.

**Work Experience:**
- Senior Sales Associate at ABC Retail (2020–Present)
- Sales Executive at XYZ Pvt Ltd (2016–2020)

**Education:** BBA in Sales and Marketing

**Skills:** Client Acquisition, Negotiation, CRM Tools, Relationship Management

"""

get_prediction(sales)

'APPAREL'

In [196]:
accountant = """
**Resume**

**Name:** Neha Agarwal
**Email:** neha.agarwal@example.com

**Professional Summary:**
Certified accountant with 8+ years of experience in auditing, tax preparation, and financial reporting. Adept at using Tally ERP and QuickBooks.

**Work Experience:**
- Senior Accountant at PwC (2018–Present)
- Accountant at Local CA Firm (2013–2018)

**Education:** B.Com in Accounting, Chartered Accountant Certification

**Skills:** Taxation, Auditing, Tally ERP, QuickBooks, Financial Reporting
"""

get_prediction(accountant)

'ACCOUNTANT'